In [212]:
#from multiprocessing import Pool
#from functools import partial
import numpy as np
#from numba import jit

In [2]:
#For regression, boston dataset
from sklearn import datasets
boston = datasets.load_boston()
X = boston.data
y = boston.target

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

/opt/anaconda3/envs/ee511/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing

In [200]:
#For classification, credit g dataset
# load data
from sklearn.datasets import fetch_openml
X, y = fetch_openml('credit-g', version=1, return_X_y=True, data_home='credit/')
X = np.array(X)
y = np.array(list(map(lambda x: 1 if x == 'good' else 0, y)))

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

In [213]:
# TODO: GBDT classification on breast cancer dataset

# load data
from sklearn import datasets
breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

In [214]:
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return X[idxs], y[idxs], idxs

train, target, idxsi = bootstrap_sample(X_train, y_train)


In [215]:
def find_split_point(train, target, lmda=0, gamma=0):
    node =None
    best_feature = None
    best_val = None
    best_gain = 0
    num_featuress = train.shape[1]
    num_features = np.random.choice(num_featuress, size=int(0.5*num_featuress),replace=True)
    for feature in (num_features):
        feature_values = train[:, feature]
        feature_values_sorted = np.sort(feature_values)
        num_sorted = feature_values_sorted.shape[0]
        thresh = []
        if type(feature_values)==[int, float]:

            for j in range(num_sorted-1):
                thresh_val = (feature_values_sorted[j] + feature_values_sorted[j+1]) / 2
                thresh.append(thresh_val)

        else: 
            for j in range(train.shape[0]):
                thresh.append(feature_values[j])
        for val in thresh:
            left_child = {'X': [], 'y': []}
            right_child = {'X': [], 'y': []}
            left_indices =  np.where(feature_values<= val)[0]
            right_indices = np.where(feature_values > val)[0]
            
            if np.sum(left_indices) == 0 or np.sum(right_indices) == 0:
                continue
            left_target=[]
            #left_x = []
            right_target=[]
            #right_x = []
            left_target = target[left_indices]
            left_child['X'] = train[left_indices]
            left_child['y'] = target[left_indices]
            
            right_target = target[right_indices]
            right_child['X'] = train[right_indices]
            right_child['y'] = target[right_indices]
        
            G_left, H_left = compute_loss_derivatives(left_target)
            G_right, H_right = compute_loss_derivatives(right_target)
            gain = ((((G_left**2)/(H_left + lmda)) + ((G_right**2)/(H_right + lmda))
                    - (((G_left + G_right)**2)/(H_left + H_right + lmda))) / 2) - gamma
            
            if gain > best_gain:
                best_gain = gain
                left_child['X'] = np.array(left_child['X'])
                right_child['X'] = np.array(right_child['X'])
                node = {'gain': gain,
                    'left_child': left_child,
                    'right_child': right_child,
                    'split_point': val,
                    'feature_idx': feature}
            if gain < 0:
                break
    return node       
#    return best_feature, best_val

def compute_g_h(target):
    g = -2*(target - np.mean(target))
    h = 2 #*(np.ones_like(target))
    return g,h


def compute_g_h_classification(target):
    predicted = 1/ (1+np.exp(-target))
    g = predicted - target  # first-order derivative
    h = predicted * (1 - predicted)  # second-order derivative
    return g, h

# def compute_g_h_classification_gbdt(target, residual):
#     predicted = 1/ (1+np.exp(-target))
#     #g = predicted - residual  # first-order derivative
#     #h = predicted * (1 - predicted)  # second-order derivative
#     g = (np.exp(predicted) / (1+np.exp(predicted))) - target
#     h = (np.exp(predicted))/((1+np.exp(predicted))**2)
#     return g, h

def compute_loss_derivatives(target):
    g, h = compute_g_h_classification(target)
    G = np.sum(g)
    H = np.sum(h)
    return G, H

# get a bootstrap sample
train, target, idxsi = bootstrap_sample(X_train, y_train)

# find the best split
best_feature = find_split_point(train, target)


In [216]:
def split_node(node, min_samples_split, max_depth, depth):
    
    left_child = node['left_child']
    right_child = node['right_child']
    
    if (left_child == None ):
        del(node['left_child'])
        node['left_split'] = terminal_node(right_child['y'])
    
    if (right_child == None ):
        del(node['right_child'])
        node['right_split'] = terminal_node(left_child['y'])
    
        #del(node['right_child'])
    del(node['left_child'])
    del(node['right_child'])
    
    if len(left_child['y']) == 0 or len(right_child['y']) == 0:
        empty_child = {'y': left_child['y'] + right_child['y']}
        node['left_split'] = terminal_node(empty_child['y'])
        node['right_split'] = terminal_node(empty_child['y'])
        return

    if depth >= max_depth:
        node['left_split'] = terminal_node(left_child['y'])
        node['right_split'] = terminal_node(right_child['y'])
        return node

    if len(left_child['y']) <= min_samples_split:
            node['left_split'] = terminal_node(left_child['y'])
    else:
        left_node = find_split_point(left_child['X'], left_child['y'])
        
        if left_node is None:
            node['left_split'] = terminal_node(left_child['y'])
        else:
            node['left_split'] = left_node
            split_node(node['left_split'], min_samples_split, max_depth, depth+1)

    if len(right_child['y']) <= min_samples_split:
            node['right_split'] = terminal_node(right_child['y'])
    else:
        right_node = find_split_point(right_child['X'], right_child['y'])
        if right_node is None:
            node['right_split'] = terminal_node(right_child['y'])
        else:
            node['right_split'] = right_node
            split_node(node['right_split'], min_samples_split, max_depth, depth+1)
    return node
    
def terminal_node(target):
    G, H = compute_loss_derivatives(target)
    weight = -G / (H+lmda)
    return weight


In [217]:
def build_tree(train, target, max_depth, min_samples_split):
    root_node = find_split_point(train, target)
    split_node(root_node, min_samples_split, max_depth, 1)
    return root_node

In [218]:
def random_forest(X_train, y_train, tmax, max_depth, min_samples_split):
    tree_ls = list()
    for i in range(tmax):
        train, target, idxsi = bootstrap_sample(X_train, y_train)
        tree = build_tree(train, target, max_depth, min_samples_split)
        tree_ls.append(tree)
    return tree_ls


In [219]:
def predict_tree(tree, X_test):
    feature_idx = tree['feature_idx']

    if X_test[feature_idx] <= tree['split_point']:
        if type(tree['left_split']) == dict:
            return predict_tree(tree['left_split'], X_test)
        else:
            value = tree['left_split']
            return value
    else:
        if type(tree['right_split']) == dict:
            return predict_tree(tree['right_split'], X_test)
        else:
            return tree['right_split']

In [220]:
def predict_rf(tree_ls, X_test):
    pred_final = []
    for i in range(len(X_test)):
        pred_ls = 0
        for tree in tree_ls:
            pred = predict_tree(tree, X_test[i])
            pred_ls +=pred
        pred_ls = pred_ls/tmax
        pred_ls = 1/(1+np.exp(-pred_ls))
        if pred_ls<=0.5:
            pred_onehot = 0
        else:
            pred_onehot = 1
            
        pred_final.append(pred_onehot)
    return pred_final

In [221]:
tmax = 28 # 5 to 50
max_depth = 6 # 2 to 10
min_samples_split = 25 # 1 to 50
lmda = 5 # 0 to 10
gamma = 0.4 # 0 to 1

model = random_forest(X_train, y_train, tmax, max_depth, min_samples_split)

In [222]:
preds = predict_rf(model, X_test)
print(preds)
print(len(preds))
print((y_test))

[1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0]
171
[1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 0 0 0 0 1 1 1 0 0 0 1 1 1 1 0 0 0 1 0
 0 1 0 1 0 0 1 0 1 1 0 0 0 1 0 1 0 1 0 0 1 1 1 0 0 1 1 1 1 1 0 1 1 0 1 0 1
 0 1 0 1 1 1 0 0 1 0 1 1 1 0 1 0 1 1 0 1 1 0 1 1 1 0 1 0 1 0 1 0 1 0 1 1 1
 1 1 1 0 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 1 1 1 0 0 0 0 1 1 1 1 1
 1 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 1 1 1 1 0 0]


In [223]:
def root_mean_square_error(pred, y):
    #TODO
    rmse = np.sqrt((np.sum((y - pred)**2))*(1/len(y)))
    return rmse

rmse = root_mean_square_error(preds, y_test)
print(rmse)

preds_train = predict_rf(model, X_train)

rmse_train = root_mean_square_error(preds_train,y_train)
print("rmse_train: ",rmse_train)

0.1873171623163388
rmse_train:  0.1736397240519698


In [224]:
# TODO: Evaluation functions (you can use code from previous homeworks)

# accuracy
def accuracy(y_pred, y_true):
    #TODO
    num_correct = 0
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i]:
            num_correct += 1
    accuracy = num_correct / len(y_true)
    return accuracy

# calculate accuracy
accuracy_test_rf = accuracy(preds,y_test)
print("Test accuracy (Random Forest): ", accuracy_test_rf)
accuracy_train_rf = accuracy(preds_train,y_train)
print("Training accuracy (Random Forest): ", accuracy_train_rf)

def precision(y_true, y_pred):
    true_positives = 0
    false_positives = 0
    for i in range(len(y_true)):
        if y_pred[i] == 1 and y_true[i] == 1:
            true_positives += 1
        elif y_pred[i] == 1 and y_true[i] == 0:
            false_positives += 1
    if true_positives + false_positives == 0:
        return 0
    else:
        return true_positives / (true_positives + false_positives)

# Calculate precision on the training and testing datasets
train_precision = precision(y_train, preds_train)
test_precision = precision(y_test, preds)

print("Training precision:", train_precision)
print("Testing precision:", test_precision)

Test accuracy (Random Forest):  0.9649122807017544
Training accuracy (Random Forest):  0.9698492462311558
Training precision: 0.9838709677419355
Testing precision: 0.9714285714285714


In [146]:
################################## GBDT ###########################################

def find_split_point_gbdt(train, target,residual, lmda=0, gamma=0):
    node =None
    best_feature = None
    best_val = None
    best_gain = 0
    num_featuress = train.shape[1]
    num_features = np.random.choice(num_featuress, size=int(0.5*num_featuress),replace=True)
    #for feature in enumerate(selected_features):
    for feature in (num_features):
        feature_values = train[:, feature]
        feature_values_sorted = np.sort(feature_values)
        num_sorted = feature_values_sorted.shape[0]
        thresh = []
        if type(feature_values)==[int, float]:

            for j in range(num_sorted-1):
                thresh_val = (feature_values_sorted[j] + feature_values_sorted[j+1]) / 2
                thresh.append(thresh_val)

        else: 
            for j in range(train.shape[0]):
                thresh.append(feature_values[j])
        for val in thresh:
            left_child = {'X': [], 'y': []}
            right_child = {'X': [], 'y': []}
            left_indices =  np.where(feature_values<= val)[0]
            right_indices = np.where(feature_values > val)[0]
            
            if np.sum(left_indices) == 0 or np.sum(right_indices) == 0:
                continue
            left_target=[]
            #left_x = []
            right_target=[]
            #right_x = []
            left_target = target[left_indices]
            left_child['X'] = train[left_indices]
            left_child['y'] = target[left_indices]
            
            right_target = target[right_indices]
            right_child['X'] = train[right_indices]
            right_child['y'] = target[right_indices]
            residual_left = residual[left_indices]
            residual_right = residual[right_indices]
        
            G_left, H_left = compute_loss_derivatives_gbdt(left_target, residual_left)
            G_right, H_right = compute_loss_derivatives_gbdt(right_target, residual_right)
            gain = ((((G_left**2)/(H_left + lmda)) + ((G_right**2)/(H_right + lmda))
                    - (((G_left + G_right)**2)/(H_left + H_right + lmda))) / 2) - gamma
            
            if gain > best_gain:
                best_gain = gain
                left_child['X'] = np.array(left_child['X'])
                right_child['X'] = np.array(right_child['X'])
                node = {'gain': gain,
                    'left_child': left_child,
                    'right_child': right_child,
                    'split_point': val,
                    'feature_idx': feature,
                    'left_indices': left_indices,
                    'right_indices': right_indices}
            if gain < 0:
                break
    return node       
#    return best_feature, best_val

def compute_g_h_gbdt(target,residual):
    g = -2*(target - residual)
    h = 2 
    return g,h


def compute_g_h_classification_gbdt(target, residual):
    predicted = 1/ (1+np.exp(-target))
    #g = predicted - residual  # first-order derivative
    #h = predicted * (1 - predicted)  # second-order derivative
    g = (np.exp(predicted) / (1+np.exp(predicted))) - target
    h = (np.exp(predicted))/((1+np.exp(predicted))**2)
    return g, h

def compute_loss_derivatives_gbdt(target,residual):
    g, h = compute_g_h_classification_gbdt(target,residual)
    # g,h = compute_g_h_gbdt(target,residual)
    G = np.sum(g)
    H = np.sum(h)
    return G, H


In [147]:
def split_node_gbdt(node, residual, min_samples_split, max_depth, depth):
    
    left_child = node['left_child']
    right_child = node['right_child']
    left_indices = node['left_indices']
    
    right_indices = node['right_indices']
    
    if (left_child == None ):
        del(node['left_child'])
        node['left_split'] = terminal_node_gbdt(right_child['y'], residual[right_indices])
    
    if (right_child == None):
        del(node['right_child'])
        node['right_split'] = terminal_node_gbdt(left_child['y'], residual[left_indices])
    
    del(node['left_child'])
    del(node['right_child'])
    del(node['left_indices'])
    del(node['right_indices'])
    
    if len(left_child['y']) == 0 or len(right_child['y']) == 0:
        empty_child = {'y': left_child['y'] + right_child['y']}
        node['left_split'] = terminal_node_gbdt(empty_child['y'],residual[left_indices])
        node['right_split'] = terminal_node_gbdt(empty_child['y'],residual[right_indices])
        return

    if depth >= max_depth:
        node['left_split'] = terminal_node_gbdt(left_child['y'],residual[left_indices])
        node['right_split'] = terminal_node_gbdt(right_child['y'],residual[right_indices])
        return node

    if len(left_child['y']) <= min_samples_split:
            node['left_split'] = terminal_node_gbdt(left_child['y'],residual[left_indices])
    else:
        left_node = find_split_point_gbdt(left_child['X'], left_child['y'], residual[left_indices])
        
        if left_node is None:
            node['left_split'] = terminal_node_gbdt(left_child['y'], residual[left_indices])
        else:
            node['left_split'] = left_node
            split_node_gbdt(node['left_split'],residual[left_indices], min_samples_split, max_depth, depth+1)

    if len(right_child['y']) <= min_samples_split:
            node['right_split'] = terminal_node_gbdt(right_child['y'],residual[right_indices])
    else:
        right_node = find_split_point_gbdt(right_child['X'], right_child['y'],residual[right_indices])
        if right_node is None:
            node['right_split'] = terminal_node_gbdt(right_child['y'],residual[right_indices])
        else:
            node['right_split'] = right_node
            split_node_gbdt(node['right_split'],residual[right_indices], min_samples_split, max_depth, depth+1)
    return node
    
def terminal_node_gbdt(target,residual):
    G, H = compute_loss_derivatives_gbdt(target,residual)
    weight = -G / (H+lmda)
    return weight


In [148]:
#### write
def build_tree_gbdt(train, target, residual, max_depth, min_samples_split):
    root_node = find_split_point_gbdt(train, target, residual)
    split_node_gbdt(root_node, residual, min_samples_split, max_depth, 1)
    return root_node

In [149]:
### check
def gbdt(X_train, y_train, tmax, max_depth, min_samples_split, learning_rate):
    #residual = y_train.copy()
    #residual = np.zeros(len(y_train), dtype=np.float64)
    tree_ls = []
    for i in range(tmax):
        if i == 0:
            residual_0 = np.mean(y_train)
            #residual = np.float64(np.full_like(y_train, residual_0))
            residual_0 = np.float64(residual_0)
            residual = np.ones(y_train.size) * residual_0
            tree = build_tree_gbdt(X_train,y_train, residual, max_depth, min_samples_split)
            tree_ls.append(tree)
            for j in range(len(X_train)):
                update = predict_tree_gbdt(tree, X_train[j]) #####
                update = np.float64(update)
            #print(residual)
            residual += learning_rate * update
            
        else:
            tree = build_tree_gbdt(X_train,y_train, residual, max_depth, min_samples_split)
            tree_ls.append(tree)
            for j in range(len(X_train)):
                update = predict_tree_gbdt(tree, X_train[j]) #######
                update = np.float64(update)
            residual += learning_rate * update
    return tree_ls

In [150]:
##### check
def predict_tree_gbdt(tree, X_test):
    feature_idx = tree['feature_idx']

    if X_test[feature_idx] <= tree['split_point']:
        if type(tree['left_split']) == dict:
            return predict_tree_gbdt(tree['left_split'], X_test)
        else:
            value = tree['left_split']
            return value
    else:
        if type(tree['right_split']) == dict:
            return predict_tree_gbdt(tree['right_split'], X_test)
        else:
            return tree['right_split']

In [151]:
#### have to change
def predict_gbdt(tree_ls, X_test):
    pred_final = []
    for i in range(len(X_test)):
        pred_ls = []
        for tree in tree_ls:
            pred = predict_tree_gbdt(tree, X_test[i])
            pred_ls.append(pred)
        #pred_ls = (pred_ls/tmax)
        pred_ls = sum(pred_ls)
        #print("pred_ls before sigmoid: ",pred_ls)
        pred_ls = 1/(1+np.exp(-pred_ls))
        if pred_ls<=0.5:
            pred_onehot = 0
        else:
            pred_onehot = 1
        pred_final.append(pred_onehot)
    return pred_final

In [152]:
tmax = 25 # 5 to 50
max_depth = 7 # 2 to 10
min_samples_split = 30 # 1 to 50
lmda = 6 # 0 to 10
gamma = 0.5 # 0 to 1
learning_rate = np.float64(0.5) # 0.1 to 1.0

model_gbdt = gbdt(X_train, y_train, tmax, max_depth, min_samples_split, learning_rate)

In [153]:
preds_gbdt = predict_gbdt(model_gbdt, X_test)
print(preds_gbdt)
print(len(preds_gbdt))
print(y_test)

[1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1]
300
[1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 0 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1
 1 0 1 1 0 1 1 0 0 1

In [154]:
#RMSE
def root_mean_square_error(pred, y):
    #TODO
    rmse = np.sqrt((np.sum((y - pred)**2))*(1/len(y)))
    return rmse

rmse_test_gbdt = root_mean_square_error(preds_gbdt, y_test)
print("rmse_test:",rmse_test_gbdt)

predsgbdt_train = predict_gbdt(model_gbdt, X_train)

rmsegbdt_train = root_mean_square_error(predsgbdt_train,y_train)
print("rmse_train: ",rmsegbdt_train)

rmse_test: 0.5354126134736337
rmse_train:  0.42928512003762054


In [155]:
# TODO: Evaluation functions (you can use code from previous homeworks)

# accuracy
def accuracy(y_pred, y_true):
    #TODO
    num_correct = 0
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i]:
            num_correct += 1
    accuracy = num_correct / len(y_true)
    return accuracy
    
# Assuming you have a classification model and test data loaded
#y_true = test_labels  # true labels of the test set
#y_pred = my_model.predict(test_data)  # predicted labels of the test set

# calculate accuracy
accuracy_test = accuracy(preds_gbdt,y_test)
print("Test accuracy: ", accuracy_test)
accuracy_train = accuracy(predsgbdt_train,y_train)
print("Training accuracy: ", accuracy_train)

def precision(y_true, y_pred):
    true_positives = 0
    false_positives = 0
    for i in range(len(y_true)):
        if y_pred[i] == 1 and y_true[i] == 1:
            true_positives += 1
        elif y_pred[i] == 1 and y_true[i] == 0:
            false_positives += 1
    if true_positives + false_positives == 0:
        return 0
    else:
        return true_positives / (true_positives + false_positives)

# Calculate precision on the training and testing datasets
train_precision = precision(y_train, predsgbdt_train)
test_precision = precision(y_test, preds_gbdt)

print("Training precision:", train_precision)
print("Testing precision:", test_precision)

Test accuracy:  0.7133333333333334
Training accuracy:  0.8157142857142857
Training precision: 0.9247058823529412
Testing precision: 0.8444444444444444


In [69]:
# TODO: GBDT classification on credit-g dataset

# load data
from sklearn.datasets import fetch_openml
X, y = fetch_openml('credit-g', version=1, return_X_y=True, data_home='credit/')
y = np.array(list(map(lambda x: 1 if x == 'good' else 0, y)))

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

In [156]:
# TODO: GBDT classification on breast cancer dataset

# load data
from sklearn import datasets
breast_cancer = datasets.load_breast_cancer()
X = breast_cancer.data
y = breast_cancer.target

# train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=8)

In [157]:
tmax = 25 # 5 to 50
max_depth = 7 # 2 to 10
min_samples_split = 30 # 1 to 50
lmda = 6 # 0 to 10
gamma = 0.5 # 0 to 1
learning_rate = np.float64(0.5) # 0.1 to 1.0

model_gbdt = gbdt(X_train, y_train, tmax, max_depth, min_samples_split, learning_rate)

In [158]:
preds_gbdt = predict_gbdt(model_gbdt, X_test)
print(preds_gbdt)
print(len(preds_gbdt))
print(y_test)

[1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0]
171
[1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 0 0 0 0 1 1 1 0 0 0 1 1 1 1 0 0 0 1 0
 0 1 0 1 0 0 1 0 1 1 0 0 0 1 0 1 0 1 0 0 1 1 1 0 0 1 1 1 1 1 0 1 1 0 1 0 1
 0 1 0 1 1 1 0 0 1 0 1 1 1 0 1 0 1 1 0 1 1 0 1 1 1 0 1 0 1 0 1 0 1 0 1 1 1
 1 1 1 0 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 0 1 0 1 0 1 1 1 0 0 0 0 1 1 1 1 1
 1 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 1 1 1 1 0 0]


In [159]:
#RMSE
def root_mean_square_error(pred, y):
    #TODO
    rmse = np.sqrt((np.sum((y - pred)**2))*(1/len(y)))
    return rmse

rmse_test_gbdt = root_mean_square_error(preds_gbdt, y_test)
print("rmse_test:",rmse_test_gbdt)

predsgbdt_train = predict_gbdt(model_gbdt, X_train)

rmsegbdt_train = root_mean_square_error(predsgbdt_train,y_train)
print("rmse_train: ",rmsegbdt_train)

rmse_test: 0.1873171623163388
rmse_train:  0.15037641213512565


In [160]:
# TODO: Evaluation functions (you can use code from previous homeworks)

# accuracy
def accuracy(y_pred, y_true):
    #TODO
    num_correct = 0
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i]:
            num_correct += 1
    accuracy = num_correct / len(y_true)
    return accuracy
    
# Assuming you have a classification model and test data loaded
#y_true = test_labels  # true labels of the test set
#y_pred = my_model.predict(test_data)  # predicted labels of the test set

# calculate accuracy
accuracy_test = accuracy(preds_gbdt,y_test)
print("Test accuracy: ", accuracy_test)
accuracy_train = accuracy(predsgbdt_train,y_train)
print("Training accuracy: ", accuracy_train)

def precision(y_true, y_pred):
    true_positives = 0
    false_positives = 0
    for i in range(len(y_true)):
        if y_pred[i] == 1 and y_true[i] == 1:
            true_positives += 1
        elif y_pred[i] == 1 and y_true[i] == 0:
            false_positives += 1
    if true_positives + false_positives == 0:
        return 0
    else:
        return true_positives / (true_positives + false_positives)

# Calculate precision on the training and testing datasets
train_precision = precision(y_train, predsgbdt_train)
test_precision = precision(y_test, preds_gbdt)

print("Training precision:", train_precision)
print("Testing precision:", test_precision)

Test accuracy:  0.9649122807017544
Training accuracy:  0.9773869346733668
Training precision: 0.9879518072289156
Testing precision: 0.9714285714285714
